In [6]:
import re
from datetime import datetime

def calcular_edad(fecha_nacimiento, fecha_urgencias):
    nacimiento = datetime.strptime(fecha_nacimiento, "%d/%m/%Y")
    urgencias = datetime.strptime(fecha_urgencias, "%d/%m/%Y")
    
    edad = urgencias.year - nacimiento.year
    if (urgencias.month, urgencias.day) < (nacimiento.month, nacimiento.day):
        edad -= 1
    
    return edad

def anonimizar_informe(informe):
    resultado = informe
    
    patron_fecha = re.compile(r'\b(\d{1,2})/(\d{1,2})/(\d{4})\b')
    
    fecha_nacimiento = None
    fecha_urgencias = None
    
    match_nacimiento = re.search(r'Fecha de nacimiento:\s*(\d{1,2}/\d{1,2}/\d{4})', resultado)
    if match_nacimiento:
        fecha_nacimiento = match_nacimiento.group(1)
    
    todas_fechas = patron_fecha.findall(resultado)
    if todas_fechas and len(todas_fechas) >= 2:
        fecha_urgencias = f"{todas_fechas[1][0]}/{todas_fechas[1][1]}/{todas_fechas[1][2]}"
    
    if fecha_nacimiento and fecha_urgencias:
        edad = calcular_edad(fecha_nacimiento, fecha_urgencias)
        resultado = re.sub(
            r'Fecha de nacimiento:\s*\d{1,2}/\d{1,2}/\d{4}',
            f'Edad: {edad}',
            resultado
        )
    
    resultado = re.sub(
        r'Nombre:\s*[^\n]+',
        'Nombre: PACIENTE',
        resultado
    )
    
    resultado = re.sub(
        r'(Dr\.|Dra\.)\s+[A-Za-z]+(\s+[A-Za-z]+)*',
        'MEDICO',
        resultado
    )
    
    resultado = patron_fecha.sub('FECHA', resultado)
    
    return resultado


informe = """Informe clinico de Urgencias
Nombre: Juan Perez Lopez
Genero: Hombre
Fecha de nacimiento: 12/05/1980
Motivo de consulta: dolor abdominal
12/05/2023 Dr. Ramirez
Tratamiento: reposo y analgesicos"""

print("INFORME ORIGINAL:")
print(informe)

print("\n\nINFORME ANONIMIZADO:")
informe_anonimizado = anonimizar_informe(informe)
print(informe_anonimizado)

print("\n\nRESULTADO ESPERADO:")
print("""Informe clinico de Urgencias
Nombre: PACIENTE
Genero: Hombre
Edad: 43
Motivo de consulta: dolor abdominal
FECHA MEDICO
Tratamiento: reposo y analgesicos""")

INFORME ORIGINAL:
Informe clinico de Urgencias
Nombre: Juan Perez Lopez
Genero: Hombre
Fecha de nacimiento: 12/05/1980
Motivo de consulta: dolor abdominal
12/05/2023 Dr. Ramirez
Tratamiento: reposo y analgesicos


INFORME ANONIMIZADO:
Informe clinico de Urgencias
Nombre: PACIENTE
Genero: Hombre
Edad: 43
Motivo de consulta: dolor abdominal
FECHA MEDICO: reposo y analgesicos


RESULTADO ESPERADO:
Informe clinico de Urgencias
Nombre: PACIENTE
Genero: Hombre
Edad: 43
Motivo de consulta: dolor abdominal
FECHA MEDICO
Tratamiento: reposo y analgesicos
